In [28]:
import pandas as pd
from kiteconnect import KiteConnect

KiteAPIKey = "b2w0sfnr1zr92nxm"
KiteAPISecret = "jtga2mp2e5fn29h8w0pe2kb722g3dh1q"
kite = KiteConnect(api_key=KiteAPIKey)

f = open("kite_data/access_token.txt", mode="r")
access_token = f.readlines()
kite.set_access_token(access_token[0])
f.close()

%run "KiteConnect_Library.ipynb"  

In [29]:
#pd.read_hdf("kite_data")
pd.HDFStore("kite_data/kite_data.h5").close()

In [30]:
instruments_df = pd.read_hdf('kite_data/kite_data.h5', key="NSE", mode='a')

EQSYMBOL = lambda x:instruments_df[instruments_df['instrument_token']==x].index[0]
EQTOKEN = lambda x:instruments_df.loc[x,'instrument_token']

In [68]:
exchange = "NSE"
symbol = "HDFC"
freq = "day"

key = freq+"/"+exchange+"/"+symbol

In [126]:
temp_data = downloadData(symbol,  fromDate = dt.datetime(2019,4,10), toDate = dt.datetime(2019,4,12), freq = freq)
temp_data.to_hdf("kite_data/kite_cache.h5", key=key, mode="a", format="table")

In [127]:
symbol = "SBIN"
key = freq+"/"+exchange+"/"+symbol
temp_data = downloadData(symbol,  fromDate = dt.datetime(2019,4,10), toDate = dt.datetime(2019,4,12), freq = freq)
temp_data.to_hdf("kite_data/kite_cache.h5", key=key, mode="a", format="table")

In [128]:
temp_data

,close,high,low,open,volume
date,,,,,
2019-04-10,310.90,317.05,310.0,314.85,19237974
2019-04-11,314.90,315.40,308.7,311.00,14897935
2019-04-12,315.25,316.40,311.2,314.60,14751964


In [129]:
rDate = pd.HDFStore("kite_data/kite_cache.h5").get('/day/NSE/SBIN').tail(1).index

In [130]:
lDate = pd.HDFStore("kite_data/kite_cache.h5").get('/day/NSE/SBIN').head(1).index

In [131]:
lDate, rDate

(DatetimeIndex(['2019-04-10'], dtype='datetime64[ns]', name='date', freq='D'),
 DatetimeIndex(['2019-04-12'], dtype='datetime64[ns]', name='date', freq='D'))

In [198]:
symbol = "SBIN"
key = freq+"/"+exchange+"/"+symbol

temp_data = downloadData(symbol,  fromDate = dt.datetime(2019,4,10), toDate = dt.datetime(2019,4,12), freq = freq)
temp_data.to_hdf("kite_data/kite_cache.h5", key=key, mode="a", format="table")

In [199]:
temp_data

,close,high,low,open,volume
date,,,,,
2019-04-10,310.90,317.05,310.0,314.85,19237974
2019-04-11,314.90,315.40,308.7,311.00,14897935
2019-04-12,315.25,316.40,311.2,314.60,14751964


New data will be downloaded if:
1. fromDate < lDate and toDate < lDate                        : download fromDate to lDate
2. fromDate < lDate and toDate >= lDate and toDate <= rDate   : download fromDate to lDate
3. fromDate >= lDate and toDate <= rDate                      : No Download
4. fromDate >=lDate and toDate > rDate and fromDate <= rDate  : download rDate to toDate
5. fromDate > rDate and toDate > rDate                        : download rDate to toDate
6. fromDate < lDate and toDate > rDate                        : download fromDate to lDate and rDate to toDate

In [210]:
symbol = "SBIN"
key = freq+"/"+exchange+"/"+symbol

fromDate = dt.datetime(2019,4,8)
toDate = dt.datetime(2019,4,10)
rDate = pd.HDFStore("kite_data/kite_cache.h5").get(key).tail(1).index
lDate = pd.HDFStore("kite_data/kite_cache.h5").get(key).head(1).index

raw_data = pd.read_hdf("kite_data/kite_cache.h5", key=key, mode="a", format="table")

if   (fromDate < lDate ) and (toDate <= rDate):
    print("Downloading data from fromDate to lDate")
    temp_data = downloadData(symbol,  fromDate, lDate, freq)
    temp_data = temp_data.append(raw_data.tail(-1))
    temp_data.to_hdf("kite_data/kite_cache.h5", key=key, mode="a", format="table")
elif (fromDate >=lDate ) and (toDate <= rDate):
    print("Not downloading data")
elif (fromDate >= lDate ) and (toDate > rDate):
    print("Downloading data from rDate to toDate")
    temp_data = downloadData(symbol,  rDate, toDate, freq)
    temp_data = raw_data.append(temp_data.tail(-1))
    temp_data.to_hdf("kite_data/kite_cache.h5", key=key, mode="a", format="table")
elif (fromDate < lDate ) and (toDate > rDate):
    print("Downloading data from fromDate to lDate")
    temp_data = downloadData(symbol,  fromDate, lDate, freq)
    temp_data = temp_data.append(raw_data.tail(-1))
    print("Downloading data from rDate to toDate")
    temp_data2 = downloadData(symbol,  rDate, toDate, freq)
    temp_data = temp_data.append(temp_data2.tail(-1))
    temp_data.to_hdf("kite_data/kite_cache.h5", key=key, mode="a", format="table")


Not downloading data


In [211]:
temp_data[(temp_data.index >= fromDate) & (temp_data.index <= toDate)]

,close,high,low,open,volume
date,,,,,
2019-04-08,312.80,319.25,311.1,318.25,17394139
2019-04-09,314.75,315.70,308.1,313.95,25556039
2019-04-10,310.90,317.05,310.0,314.85,19237974


In [152]:
raw_data.tail(-1)

,close,high,low,open,volume
date,,,,,
2019-04-11,314.90,315.4,308.7,311.0,14897935
2019-04-12,315.25,316.4,311.2,314.6,14751964


In [156]:
temp_data.head(-2)

,close,high,low,open,volume
date,,,,,
2019-04-01,322.75,327.65,321.05,321.80,22372357
2019-04-02,329.00,331.00,322.10,323.65,27897583
2019-04-03,320.50,332.45,319.50,331.00,30552104
2019-04-04,321.80,325.75,318.30,321.50,27832294
2019-04-05,317.05,323.70,315.00,323.70,23117232
2019-04-08,312.80,319.25,311.10,318.25,17394139
2019-04-09,314.75,315.70,308.10,313.95,25556039
2019-04-10,310.90,317.05,310.00,314.85,19237974
2019-04-10,310.90,317.05,310.00,314.85,19237974
